In [ ]:
import pandas as pd
import numpy as np
from keras.utils.np_utils import to_categorical
from keras import models
from keras import layers

train = pd.read_csv('../input/Kannada-MNIST/train.csv')
test = pd.read_csv('../input/Kannada-MNIST/test.csv')

In [ ]:
y_train = train["label"]
#remove label column from X_train
x_train = train.drop(labels = ["label"],axis = 1)
x_train = x_train.to_numpy().reshape(-1,28,28,1)
y_train = to_categorical(y_train, num_classes = 10)

# remove id column from test data
x_test = test.copy()
x_test.drop(labels=['id'], axis=1, inplace=True)
x_test = x_test.to_numpy().reshape(-1,28,28,1)


In [ ]:
print(x_train.shape)

In [ ]:
#rescale values to range [0, 1]
x_train = x_train / 255.0
x_test = x_test / 255.0

In [ ]:
# x_additional = pd.read_csv('../input/Kannada-MNIST/Dig-MNIST.csv')
# # prepare the additional data set
# y_additional = x_additional['label']
# x_additional.drop(labels=['label'], axis=1, inplace=True)
# x_additional = x_additional.to_numpy().reshape(-1, 28, 28, 1)
# y_additional = to_categorical(y_additional, num_classes = 10)


In [ ]:
# #prepare additional data
# x_additional = x_additional / 255.0

In [ ]:
# #concatenate training data and additional data
# x_train = np.concatenate((x_train, x_additional), axis=0)
# y_train = np.concatenate((y_train, y_additional), axis=0)

In [ ]:
from keras.preprocessing.image import ImageDataGenerator

train_datagen = ImageDataGenerator(
    rotation_range=10,
    width_shift_range=0.1,
    height_shift_range=0.1, 
#     shear_range=0.1, 
    zoom_range=0.1)

In [ ]:
model_list = []

model_1 = models.Sequential()
model_1.add(layers.Conv2D(10, (5, 5), input_shape = (28, 28, 1)))
model_1.add(layers.BatchNormalization())
model_1.add(layers.Activation('relu'))
model_1.add(layers.MaxPooling2D((2, 2)))
model_1.add(layers.Conv2D(20, (5, 5)))
model_1.add(layers.BatchNormalization())
model_1.add(layers.Activation('relu'))
model_1.add(layers.MaxPooling2D((2, 2)))
model_1.add(layers.Flatten())
model_1.add(layers.Dropout(0.5))
model_1.add(layers.Dense(100))
model_1.add(layers.BatchNormalization())
model_1.add(layers.Activation('relu'))
model_1.add(layers.Dense(10, activation = 'softmax'))
model_list.append(model_1)


model_2 = models.Sequential()
model_2.add(layers.Conv2D(20, (3, 3), input_shape = (28, 28, 1)))
model_2.add(layers.BatchNormalization())
model_2.add(layers.Activation('relu'))
model_2.add(layers.MaxPooling2D((2, 2)))
model_2.add(layers.Conv2D(30, (5, 5)))
model_2.add(layers.BatchNormalization())
model_2.add(layers.Activation('relu'))
model_2.add(layers.MaxPooling2D((2, 2)))
model_2.add(layers.Flatten())
model_2.add(layers.Dropout(0.5))
model_2.add(layers.Dense(100))
model_2.add(layers.BatchNormalization())
model_2.add(layers.Activation('relu'))
model_2.add(layers.Dense(10, activation = 'softmax'))
model_list.append(model_2)

model_3 = models.Sequential()
model_3.add(layers.Conv2D(10, (3, 3), input_shape = (28, 28, 1)))
model_3.add(layers.BatchNormalization())
model_3.add(layers.Activation('relu'))
model_3.add(layers.MaxPooling2D((2, 2)))
model_3.add(layers.Conv2D(20, (5, 5)))
model_3.add(layers.BatchNormalization())
model_3.add(layers.Activation('relu'))
model_3.add(layers.MaxPooling2D((2, 2)))
model_3.add(layers.Flatten())
model_3.add(layers.Dense(100))
model_3.add(layers.BatchNormalization())
model_3.add(layers.Activation('relu'))
model_3.add(layers.Dense(10, activation = 'softmax'))
model_list.append(model_3)

In [ ]:
import keras
callbacks_lists = []
for i in range(3):
    callbacks_list = [
        keras.callbacks.EarlyStopping(
            monitor='val_accuracy',
            patience=2,
        ), 
        keras.callbacks.ModelCheckpoint(
                filepath='ensemble_model_' + str(i) + '.h5',
                monitor='val_accuracy',
                save_best_only=True,
        )
    ]
    callbacks_lists.append(callbacks_list)

In [ ]:
from keras import optimizers

for i in range(3):
    model_list[i].compile(loss='categorical_crossentropy', optimizer='adam', metrics=['accuracy'])

In [ ]:
from sklearn.model_selection import train_test_split

for i in range(3):
    #split data into training set and validation set
    x_train_2, x_val_2, y_train_2, y_val_2 = train_test_split(x_train, y_train, test_size = 0.2)
    epoch_step = x_train.shape[0]/64
    history = model_list[i].fit_generator(train_datagen.flow(x_train_2, y_train_2, batch_size=64), 
                              epochs = 50, callbacks = callbacks_lists[i], 
                              steps_per_epoch = epoch_step, validation_data = (x_val_2,y_val_2))
    print('history: ', history)

In [ ]:
from keras.models import load_model
from sklearn.metrics import accuracy_score

results = np.zeros((x_test.shape[0], 10))
# results_additional = np.zeros((x_additional.shape[0], 10))
for i in range (3):
    saved_model = load_model('ensemble_model_' + str(i) + '.h5')
    result = saved_model.predict(x_test)
    results += result
#     result_additional = saved_model.predict(x_additional)
#     results_additional += result_additional
    
best_result = np.argmax(results,axis = 1)
# best_result_additional = np.argmax(results_additional,axis = 1)

# best_additional_accuracy = accuracy_score(y_additional, best_result_additional)
# print('best_additional_accuracy', best_additional_accuracy)

final_result = pd.Series(best_result,name="label")
submission = pd.concat([pd.Series(range(0,x_test.shape[0]),name = "id"),final_result],axis = 1)
submission.to_csv("submission.csv",index=False)